In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
## Image Folder Paths
GALLERY = "/kaggle/input/database-of-faces/Gallery/Gallery"
PROBE = "/kaggle/input/database-of-faces/Probe/Probe"

In [ ]:
## Storing Images and corresponding labels
images = list()
labels = list()
for folder in os.listdir(GALLERY):
    for img in os.listdir(GALLERY + "/" + folder):
        img = Image.open(GALLERY + "/" + folder + "/" + img)
        labels.append(int(folder.split("s")[1]))
        images.append(np.array(img))

In [ ]:
# Resizing images in the form of a vector and subtracting mean from each
shape = images[0].shape
X = np.resize(images,(len(images), shape[0]*shape[1]))
mean = np.mean(X, axis = 0)
X_std = X - mean

In [ ]:
# m x d where m represents number of training samples and d represents all dimensions of the image
X_std.shape

In [ ]:
# IMPORTANT, cov matrix here is of dimension m x m instead of the standard d x d
cov_mat = X_std @ X_std.T

In [ ]:
# m x m
cov_mat.shape

In [ ]:
# Eigendecomposition of the covariance matrix followed by sorting of the eigenvectors in decreasing order of the eigenvalues
eigenvalues, eigenvectors = np.linalg.eig(cov_mat)
idx = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:,idx]

In [ ]:
# Extracting the eigenvectors of the actual covariance matrix from the converted covariance matrix
eigenvector_converted = X_std.T @ eigenvectors
eigenvector_converted.shape

In [ ]:
# Normalizing the eigenvectors (making them unit norm)
eigenfaces = eigenvector_converted.T/ np.sqrt((eigenvector_converted.T ** 2).sum(axis=1, keepdims=True))
eigenfaces.shape

In [ ]:
# Displaying the Eigenface images corresponding to the top 6 Eigenvalues of the covariance matrix Σ.
_, axs = plt.subplots(2, 3, figsize=(10, 10))
axs = axs.flatten()
for i, (img, ax) in enumerate(zip(eigenfaces, axs)):
    ax.set_title(f"Eigenvalue: {np.round(eigenvalues[i], 2)}")
    ax.imshow(img.reshape(shape[0], shape[1]), cmap="gray")
plt.show()

In [ ]:
# Last Eigenvalue is approximately 0
eigenvalues[-1]

In [ ]:
# Calculate the total sum of eigenvalues, which represent the total variance in the data
total = sum(eigenvalues)

# Calculate the explained variance for each eigenvalue as a percentage of the total variance
explained_variance = [(i / total) * 100 for i in eigenvalues]
explained_variance = np.round(explained_variance, 2)

# Calculate the cumulative explained variance by taking the cumulative sum of the explained variance values
cum_explained_variance = np.cumsum(explained_variance)

In [ ]:
# Initialize an empty list 'x' to store the dimensions that satisfy the explained variance thresholds
x = list()

# Define the desired cumulative explained variance thresholds in 'y'
y = [85, 95]

for dim in range(200):
    # Check if the cumulative explained variance for the current dimension
    # is greater than or equal to the first threshold (y[0])
    if cum_explained_variance[dim] >= y[0]:
        x.append(dim+1)
        break

for dim in range(x[0]+1, 200):
    # Check if the cumulative explained variance for the current dimension
    # is greater than or equal to the second threshold (y[1])
    if cum_explained_variance[dim] >= y[1]:
        x.append(dim+1)
        break

In [ ]:
# Number of dimensions accounting for 85% and 95% of the total variance of the original data in the reduced space
x

In [ ]:
# A graph depicting the percentage of the total variance of the original data retained in the reduced space versus the number of dimensions.
plt.plot(cum_explained_variance)
plt.plot(x, y, "o")
plt.xlabel("Number of Dimensions")
plt.ylabel("Cumulative Explained Variance")
plt.show()

In [ ]:
train_img = np.array(Image.open(GALLERY + "/" + "s1/1.pgm"))
train_image_mean_centered = train_img.reshape(shape[0]*shape[1]) - mean
plt.imshow(train_img, cmap = 'gray')
plt.show()

In [ ]:
_, axs = plt.subplots(2, 3, figsize=(10, 10))
axs = axs.flatten()
# Define a list of indices 'count' indicating the number of eigenfaces to use for reconstruction
count = [1, 4, 15, 150, 199, 200]
print("Reconstruction errors: (MSE)")
for i, (img, ax) in enumerate(zip(eigenfaces, axs)):
    # Calculate the reconstruction using the selected number of eigenfaces
    E = eigenfaces[:count[i]].dot(train_image_mean_centered)
    reconstruction = eigenfaces[:count[i]].T.dot(E) + mean
    
    # Calculate the difference between the original and reconstructed images
    diff = train_image_mean_centered + mean - reconstruction
    
    # Calculate the Mean Squared Error (MSE) as the reconstruction error
    error = np.dot(diff, diff)
    print(str(count[i]) + " eigenfaces: " + str(error))
    
    # Display the reconstructed image in grayscale
    ax.set_title(str(count[i]) + " eigenfaces")
    ax.imshow(np.reshape(reconstruction, (shape[0], shape[1])), cmap='gray')
plt.show()

In [ ]:
test_img = np.array(Image.open(PROBE + "/" + "s1/6.pgm"))
test_image_mean_centered = test_img.reshape(shape[0]*shape[1]) - mean
plt.imshow(test_img.reshape(shape[0], shape[1]), cmap = 'gray')
plt.show()

In [ ]:
_, axs = plt.subplots(2, 3, figsize=(10, 10))
axs = axs.flatten()
# Define a list of indices 'count' indicating the number of eigenfaces to use for reconstruction
count = [1, 4, 15, 150, 199, 200]
print("Reconstruction errors: (MSE)")
for i, (img, ax) in enumerate(zip(eigenfaces, axs)):
    # Calculate the reconstruction using the selected number of eigenfaces
    E = eigenfaces[:count[i]].dot(test_image_mean_centered)
    reconstruction = eigenfaces[:count[i]].T.dot(E) + mean
    
    # Calculate the difference between the original and reconstructed images
    diff = test_image_mean_centered + mean - reconstruction
    
    # Calculate the Mean Squared Error (MSE) as the reconstruction error
    error = np.dot(diff, diff)
    print(str(count[i]) + " eigenfaces: " + str(error))
    
    # Display the reconstructed image in grayscale
    ax.set_title(str(count[i])+" eigenfaces")
    ax.imshow(np.reshape(reconstruction,(shape[0],shape[1])),cmap='gray')
plt.show()

In [ ]:
errors = list()
for i in range(200):
    # Project the test image onto the first 'i' principal components
    E = eigenfaces[:i].dot(test_image_mean_centered)
    
    # Reconstruct the test image using the selected number of dimensions
    reconstruction = eigenfaces[:i].T.dot(E) + mean
    
    # Calculate the difference between the original test image and the reconstructed image
    diff = test_image_mean_centered + mean - reconstruction
    
    # Calculate the Mean Squared Error (MSE) as the reconstruction error
    error = np.dot(diff, diff)
    errors.append(error)

# Plot the MSE values against the number of dimensions used for reconstruction
plt.plot(errors)
plt.title("MSE vs Number of Dimensions")
plt.xlabel("Number of Dimensions")
plt.ylabel("Mean Squared Error")
plt.show()

In [ ]:
# Initialize empty lists to store the lower-dimensional representations (E) of test and train images
testE = list()  # For test images
trainE = list()  # For train images

# Iterate through folders in the PROBE directory (presumably containing test images)
for folder in os.listdir(PROBE):
    # Iterate through image files within each folder
    for img in os.listdir(PROBE + "/" + folder):
        # Load and convert the image to a NumPy array
        img = np.array(Image.open(PROBE + "/" + folder + "/" + img))
        
        # Compute the mean-centered image by subtracting the mean image
        img_mean_centered = img.reshape(shape[0] * shape[1]) - mean
        
        # Project the mean-centered test image onto the first 199 principal components
        testE.append(eigenfaces[:199].dot(img_mean_centered))

# Iterate through folders in the GALLERY directory (presumably containing train images)
for folder in os.listdir(GALLERY):
    # Iterate through image files within each folder
    for img in os.listdir(GALLERY + "/" + folder):
        # Load and convert the image to a NumPy array
        img = np.array(Image.open(GALLERY + "/" + folder + "/" + img))
        
        # Compute the mean-centered image by subtracting the mean image
        img_mean_centered = img.reshape(shape[0] * shape[1]) - mean
        
        # Project the mean-centered train image onto the first 199 principal components
        trainE.append(eigenfaces[:199].dot(img_mean_centered))


In [ ]:
def predictSingle(img, total, q = 25):
    smallest_value =None
    index = None 
    E = testE[total-1][:q]
    for i, image in enumerate(images):
        E_compare = trainE[i][:q]
        diff = E-E_compare
        eps = np.sqrt(np.dot(diff, diff))
        if smallest_value==None:
            smallest_value=eps
            index = labels[i]
        if smallest_value>eps:
            smallest_value=eps
            index=labels[i]
    return index

In [ ]:
def predict(q = 25):
    total = 0
    correct = 0
    for folder in os.listdir(PROBE):
        for img in os.listdir(PROBE + "/" + folder):
            img = np.array(Image.open(PROBE + "/" + folder + "/" + img))
            total += 1
            if(predictSingle(img, total, q) == int(folder.split("s")[1])):
                correct += 1
    return correct/total

In [ ]:
partA = predict()
print("Classification Accuracy: ", partA)

In [ ]:
accs = list()
for q in range(1, 200):
    accs.append(predict(q))

In [ ]:
plt.plot(accs)
plt.title("Accuracies vs Number of Dimensions")
plt.xlabel("Number of Dimensions")
plt.ylabel("Recognition Accuracies")
plt.show()